In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [ ]:
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection

# Create Index

In [ ]:
import pickle as pkl

with open('splitDocuments.pkl','rb') as f:
  all_splits = pkl.load(f)


In [ ]:
all_documents = []
all_metadata = []
for i in all_splits:
    for j in i:
        all_documents.append(j.page_content)
        all_metadata.append(j.metadata)


In [ ]:
!pip install ragatouille

In [ ]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [ ]:
RAG.index(
    collection=all_documents, 
    document_metadatas=all_metadata,
    index_name="PretrainedColbertV2", 
    max_document_length=500, 
    split_documents=False
    )

# Load Index and Run Search

In [ ]:
from ragatouille import RAGPretrainedModel


path_to_index = ".ragatouille/colbert/indexes/PretrainedColbertV2"
RAG = RAGPretrainedModel.from_index(path_to_index)


In [ ]:
all_results = RAG.search(query=["What is buggy?"], k=3)
all_results